## RMT Util

### TODO:  refactor to allow pyTorch or Keras Models



In [3]:
import sys
import pickle, time
from copy import deepcopy
from shutil import copy
from tqdm import tqdm_notebook as tqdm

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import SGD
from keras.initializers import Constant
import keras.backend as K

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.decomposition import TruncatedSVD
#from sklearn.random_projection import sparse_random_matrix
import scipy as sp
from scipy.linalg import svd

import powerlaw
from scipy import optimize
from sklearn.neighbors import KernelDensity


print(sys.version)
print("numpy version {}".format(np.__version__))
print("scipy version {}".format(sp.__version__))
print("tensforflow version {}".format(tf.__version__))
print("keras version {}".format(keras.__version__))
print("sklearn version {}".format(sklearn.__version__))
print("powerlaw version {}".format(powerlaw.__version__))


/Users/charlesmartin14/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/charlesmartin14/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


3.6.5 |Anaconda custom (64-bit)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy version 1.14.2
scipy version 1.0.1
tensforflow version 1.5.0-dev20171116
keras version 2.1.5
sklearn version 0.19.1
powerlaw version 1.4.3


### MLP3 Model

In [4]:
def MLP3_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 3)))
    model.add(Dense(512, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='relu'))
    model.add(Dense(512, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='relu'))
    model.add(Dense(512, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='relu'))
    model.add(Dense(10, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='softmax'))

    return model

### AlexNet Model

#### batchnorm = True   only for last 2 layers

In [5]:
def Alexnet_Model(batchnorm=False):
    model = Sequential()
    model.add(Conv2D(96, (5, 5), input_shape=(28, 28, 3), kernel_initializer=
                     'glorot_normal', bias_initializer=Constant(0.1), padding=
                     'same', activation='relu')) 
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (5, 5), kernel_initializer='glorot_normal',
                     bias_initializer=Constant(0.1), padding='same',
                     activation='relu')) 
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Flatten())

    model.add(Dense(384, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='relu'))
    if batchnorm:
        model.add(BatchNormalization())
    model.add(Dense(192, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='relu'))
    if batchnorm:
        model.add(BatchNormalization())

    model.add(Dense(10, kernel_initializer='glorot_normal',
                    bias_initializer=Constant(0.1), activation='softmax'))

    return model

### Generalized Entropy

In [6]:
def load_keras_matrix(model, weightfile, layer=2):
    model.load_weights(weightfile)
    return get_keras_matrix(model, layer)

In [7]:
def get_keras_matrix(model, layer=2):
    return model.layers[layer].get_weights()[0]

In [8]:
# model is defined above
def layer_entropy(model, layer=2):
    W = get_keras_matrix(model, layer) 
    return matrix_entropy(W)

In [9]:
# Trace Normalization
def matrix_entropy(W):
    W=W/np.trace(W)
    m = W.shape[1]
    #svd = TruncatedSVD(n_components=m-1, n_iter=7, random_state=10)
    #svd.fit(W) 
    #sv = svd.singular_values_
    u, sv, vh = svd(W)
    
    rank = np.linalg.matrix_rank(W)
    p = sv*sv
    p = p/np.sum(p)
    
    if (rank==1):
        rank=1.000001
    entropy = - np.sum(p*np.log(p)) / np.log(rank) 
    return entropy


### Read Training Accuracies

We can simply parse these from the output file

In [10]:
def read_final_test_accuracy(filename):
    cmd = "tail -1 ./{} | sed -e 's/\]//' |  sed -e 's/\[//' ".format(filename)
    !$cmd > tmp.out
    result = np.loadtxt("tmp.out",delimiter=",")
    !rm tmp.out
    return result[1]

In [11]:
def read_training_accuracies(filename):
    cmd = "cat ./{} | grep '50000/' | sed -e 's/.*acc://' ".format(filename)
    !$cmd > tmp.out
    result = np.loadtxt("tmp.out")
    !rm tmp.out
    return result

### Recompute Test Accuracies

In [12]:
def read_test_images(datadir="../data"):

    img = tf.placeholder(tf.float32, [28, 28, 3])
    norm_image = tf.image.per_image_standardization(img)

    print("Loading test images...")
    cifar10_test_images = []
    cifar10_test_labels = []
    datafile = "{}/cifar-10-batches-py/test_batch".format(datadir)
    
    test_file = open(datafile, 'rb')
    test_dict = pickle.load(test_file, encoding='bytes')

    for image, label in zip(test_dict[b'data'], test_dict[b'labels']):
        image_red = np.reshape(image[:1024], (32, 32))[2:-2, 2:-2] / 255.0
        image_red = np.reshape(image_red, (28, 28, 1))
        image_green = np.reshape(image[1024:2048], (32, 32))[2:-2,
                                                                2:-2] / 255.0
        image_green = np.reshape(image_green, (28, 28, 1))
        image_blue = np.reshape(image[2048:3072], (32, 32))[2:-2, 2:-2] / 255.0
        image_blue = np.reshape(image_blue, (28, 28, 1))
        image_blue = np.reshape(image_blue, (28, 28, 1))
        image = np.concatenate([image_red, image_green, image_blue], axis=-1)
        image = norm_image.eval(feed_dict={img:image})
        cifar10_test_images.append(image)
        label = np.identity(10)[label]
        cifar10_test_labels.append(label)
    test_file.close()
    
    return np.array(cifar10_test_images),np.array(cifar10_test_labels)

## Wigner SemiCircle Plots

### <font color='orange'>Not available yet </font>

## Marchenko Pastur FIts and Plots

$$\rho(\lambda)=\dfrac{Q}{2\pi\sigma_{mp}^{2}}\dfrac{(\lambda^{+}-\lambda)(\lambda^{-}-\lambda)}{\lambda}$$


$$\lambda^{\pm}=\sigma_{mp}^{2}\left(1\pm\dfrac{1}{\sqrt Q}\right)^{2}$$


In [13]:
def marchenko_pastur_pdf(x_min, x_max, Q, sigma=1):
    y=1/Q
    x=np.arange(x_min,x_max,0.001)

    b=np.power(sigma*(1 + np.sqrt(1/Q)),2) # Largest eigenvalue
    a=np.power(sigma*(1 - np.sqrt(1/Q)),2) # Smallest eigenvalue
    return x, (1/(2*np.pi*sigma*sigma*x*y))*np.sqrt((b-x)*(x-a))

#### Get sigma from $\lambda_{max}$ for rectangular matrix


$$\sigma^{2}=\lambda_{max}\left(1+\dfrac{1}{\sqrt{Q}}\right)^{-2}$$

In [14]:
def calc_sigma(Q, evs):
    lmax = np.max(evs)
    inv_sqQ = 1.0/np.sqrt(Q)
    sigma_2 = lmax/np.square(1+inv_sqQ)
    sigma = np.sqrt(sigma_2)
    return sigma

In [15]:
def calc_lambda_plus(Q, sigma):
    return np.power(sigma*(1 + np.sqrt(1/Q)),2)

In [16]:
def calc_lambda_minus(Q, sigma):
    return np.power(sigma*(1 - np.sqrt(1/Q)),2)

In [17]:
def get_Q(W):
    if W.shape[1] > W.shape[0]:
        M, N = W.shape
    else:
        N, M = W.shape
    Q=N/M
    return Q

In [18]:
def hard_rank(W, sv):
    """hard rank, using tolerance from numerical recipes, not default scipy tolerance"""
    tol = np.max(sv)*np.finfo(np.float32).eps/(2.0*np.sqrt(np.sum(W.shape)+1))
    return np.linalg.matrix_rank(W, tol = tol)

In [19]:
def get_eigenvalues(model, weightfile, layer=7):
    "Read the keras weightfile, get weights for the layer, and compute the eigenvalues(-1)"
    W = load_keras_matrix(model, weightfile, layer)
    u, sv, vh = svd(W)
    eigenvalues = sv*sv
    return eigenvalues

In [20]:
# uss FAST SVD method: notice we miss 1 eigenvalue here...using 
def get_shuffled_eigenvalues(W, layer=7, num=100):
    "get eigenvalues for this model, but shuffled, num times"
    
    print("get_shuffled_eigenvalues")
    N, M = W.shape[0], W.shape[1]       

    if (N<M):
        N, M = W.shape[1], W.shape[0] 
   
    eigenvalues = []
    for idx in tqdm(range(num)):
        W_shuf = W.flatten()
        np.random.shuffle(W_shuf)
        W_shuf = W_shuf.reshape([N,M])

        u, sv, vh = svd(W_shuf)

        eigenvalues.extend(sv*sv)
        
    evals = (np.array(eigenvalues).flatten())
    return evals

### Fit Marchenko Pastur to ESD

#### set sigma, or compute sigma from Q 

In [3]:
def plot_ESD_and_fit(eigenvalues=None, model=None,  weightfile=None, 
                     layer=2,  Q=1.0, num_spikes=0, sigma=None, 
                     alpha=0.25, color='blue', skip=False, verbose=True):
    """Plot histogram of eigenvalues, for Q, and fit Marchenk Pastur.  
    If no sigma, calculates from maximum eigenvalue (minus spikes)
    Can read keras weights from model if specified.  Does not read PyTorch"""
    if eigenvalues is None:
        eigenvalues = get_eigenvalues(model, weightfile, layer)
   
    plt.hist(eigenvalues, bins=100, alpha=alpha, color=color, density=True, label=r'$\rho_{emp}(\lambda)$');

    if skip:
        return
    if not (num_spikes):
        num_spikes = 0
        
    # sort, descending order, minus a few max eigenvalues
    evals = np.sort(eigenvalues)[::-1][num_spikes:]

    if (sigma is None):
        sigma=calc_sigma(Q,evals)
        
    percent_mass = 100.0*(num_spikes)/len(evals)

    ev = np.array(evals)
    x_min, x_max =  0, np.max(evals)

    x, mp = marchenko_pastur_pdf(x_min, x_max, Q, sigma)
    if verbose:
        print("% spikes outside bulk {0:.2f}".format(percent_mass))
        print("% sigma {0:.4f}".format(sigma))

    plt.plot(x,mp, linewidth=1, color = 'r', label="MP fit")
    plt.title(" W{} ESD, MP Sigma={}".format(layer,sigma));
    return sigma

## Scree Plots:  

In [22]:
# Eigenvalues for correlation weight matrices, for last 2 layers
# for Scree plots
# check normalization...maybe replace with svd approach above
def matrix_eigenvalues(model, layer=2):    
    W = model.layers[layer].get_weights()[0]
    W = W / np.linalg.norm(W)
    WW = np.dot(W.transpose(),W)#/float(N7)
    evs, _ = np.linalg.eig(WW)    
    return evs

In [23]:
def scree_plot(model, weightfile, layer=2, color='blue'):    
    model.load_weights(weightfile)
    evs = matrix_eigenvalues(model, layer)
    eigvals = np.flip(np.sort(evs), axis=0)
    sing_vals = np.arange(len(eigvals)) + 1
    plt.plot(sing_vals, eigvals, color, linewidth=1)

## Soft / Stable Rank

In [24]:
def stable_rank(evals):
    return np.sum(evals)/np.max(evals)

In [25]:
def layer_soft_rank(model, layer):
    W = model.layers[layer].get_weights()[0]
    return matrix_soft_rank(W)

In [26]:
def keras_layer_soft_rank(model, layer):
    W = keras_matrix(model, layer)
    return matrix_soft_rank(W)

In [27]:
def matrix_soft_rank(W):
    W=W/np.trace(W)
    u, sv, vh = svd(W)
    return stable_rank(sv*sv)

In [28]:
def mp_soft_rank(evals, num_spikes):
    evals = np.array(evals)
    lambda_max = np.max(evals)
    if num_spikes> 0:
        evals = np.sort(evals)[::-1][num_spikes:]
        lambda_plus = np.max(evals)
    else:
        lambda_plus = lambda_max
        
    return lambda_plus/lambda_max

In [29]:
def calc_mp_soft_rank(evals, Q, sigma):
    
    lambda_plus = calc_lambda_plus(Q,sigma)
    lambda_max = np.max(evals)
  
    return lambda_plus/lambda_max

### Eigenvector Localization

In [30]:
def keras_matrix(model, modelfile, layer):
    model.load_weights(modelfile)
    W = model.layers[layer].get_weights()[0]
    return W

In [31]:
def pytorch_matrix(model, modelfile, layer):
   # model.load_weights(modelfile)
   # W = model.layers[layer].get_weights()[0]
   # return W
    return None

In [32]:
def eigenspectrum(W, norm=False):
    N, M = W.shape
    if N < M:
        X = np.dot(W,W.transpose())
        
    else:
        X = np.dot(W.transpose(),W)
       
    # assumes matrix is full rank
    if norm:
        X = X/np.trace(X)
        
    return np.linalg.eig(X)

In [33]:
# TODO: replace with full svd
def singular_spectrum(W, norm=False): 
    if norm:
        W = W/np.trace(W)
    M = np.min(W.shape)
        
    svd = TruncatedSVD(n_components=M-1, n_iter=7, random_state=10)
    svd.fit(W) 
    svals = svd.singular_values_
    svecs = svd.components_
    return svals, svecs

### TODO:  fix ensemble runs

In [34]:
def keras_ensemble_spectrum(model, ensemble_weightfile, layer, num_runs):
    """compute list of arrays of eigenvals and eigenves for an ensemle run"""
    
    eigenvalues, eigenvectors = [], []
    for i in tqdm(range(1,num_runs+1)):
        filename = ensemble_weightfile.format(i)
        W = keras_matrix(model, filename, layer)
        evs, evecs = eigenspectrum(W)
        
        eigenvalues.append(evs)
        eigenvectors.append(evecs)
    return eigenvalues, eigenvectors

In [35]:
def keras_singular_spectrum(model, ensemble_weightfile, layer, num_runs):
    """compute list of arrays of singular vals and vectors for an ensemle run"""
    
    svalues, svectors = [], []
    for i in tqdm(range(1,num_runs+1)):
        filename = ensemble_weightfile.format(i)
        W = keras_matrix(model, filename, layer)
        svs, svecs = singular_spectrum(W)
        
        svalues.append(svs)
        svectors.append(svecs)
    return svalues, svectors 

In [36]:
def localization_ratio(vec):
    return np.linalg.norm(vec, ord=1) / np.linalg.norm(vec, ord=np.inf)

In [37]:
def participation_ratio(vec):
    return np.linalg.norm(vec, ord=2) / np.linalg.norm(vec, ord=4)

In [38]:
def vector_entropy(u):
    """Vector entropy, as in the  Porter-Thomas distribution
    
    see:  https://arxiv.org/pdf/cond-mat/9810255.pdf
    """
    u2 = np.linalg.norm(u)/2
    entropy = np.sum(np.exp(-u2)/np.sqrt(2.0*np.pi))
    return entropy

### TODO:  Add perEpoch Methods

- entropies
- ranks
- recompute test data


In [39]:
def entropy_per_epoch(model, per_epoch_weightfile, num_epochs, layer):
    """Computer layer entropy per epoch for a keras layer"""
    entropies = []

    for epoch in tqdm(range(num_epochs)):
        model.load_weights(per_epoch_weightfile.format(epoch))  
        entropies.append(layer_entropy(model, layer))

    return np.array(entropies)

In [40]:
def stable_rank_per_epoch(model, per_epoch_weightfile, num_epochs, layer):
    """Computer layer entropy per epoch for a keras layer"""
    ranks = []

    for epoch in tqdm(range(num_epochs)):
        model.load_weights(per_epoch_weightfile.format(epoch))  
        entropies.append(layer_stable_rank(model, layer))

    return np.array(ranks)

In [41]:
def discrete_entropy(vec, num_bins=100):
    vec = vec - np.mean(vec)
    h = np.histogram(vec, density=True, bins=num_bins)[0];
    p = np.array(h)+0.0000000001
    
    p = p / np.sqrt(2*np.pi)
    p = p / np.sum(p)

    #p = p/(2*np.pi)
    entropy = -np.sum(p*np.log(p))
    entropy = entropy#/(2*np.pi)#/float(num_bins)
    return entropy

In [42]:
def max_discrete_entropy(len_vec,num_bins=100, sample_size = 100000):
    """compute maximum possible entropy for this vector length and bin sizes"""
    entropies = []
    for i in tqdm(range(sample_size)):
        test_vec = np.random.normal(0,1,len_vec)
        s = discrete_entropy(test_vec, num_bins=num_bins)
        entropies.append(s)

    return np.max(entropies)

### Misc

In [43]:
def spike_min(Q):
    """minimum perturbation to W to see a spike"""
    return 1/np.sqrt(np.sqrt(Q))

In [44]:
def spike_lmax(S,Q):
    """Maximum spike given a perturbation"""
    S2 = S*S
    return ((1.0/Q)+S2)*(1+(1.0/S2))

### Fit PowerLaw

In [45]:
def mu4alpha(alpha):
    if (alpha >= 2.0 and alpha <= 3.0):
        mu = 2.0
    elif (alpha < 2.0):
        mu = 2.0*(alpha - 1.0)
    elif (alpha > 3.0):
        mu = alpha - 1.0
    return mu


In [46]:
def best_dist(fit):
    distName = 'power_law'
    dist = "PL"

    R, p = fit.distribution_compare('truncated_power_law', 'power_law', normalized_ratio=True)
    if R>0 and p <= 0.05:
        distName = 'truncated_power_law'
        dist = 'TPL'
        
    R, p = fit.distribution_compare(distName, 'exponential', normalized_ratio=True)
    if R<0 and p <= 0.05:
        dist = 'EXP'
        return dist

    R, p = fit.distribution_compare(distName, 'stretched_exponential', normalized_ratio=True)
    if R<0 and p <= 0.05:
        dist = 'S_EXP'
        return dist
        
    R, p = fit.distribution_compare(distName, 'lognormal', normalized_ratio=True)
    if R<0 and p <= 0.05:
        dist = 'LOG_N'
        return dist

    return dist

In [47]:
def fit_powerlaw(evals, verbose=True):
    fit = powerlaw.Fit(evals, xmax=np.max(evals))
    return [fit.alpha, fit.D, best_dist(fit)]

In [48]:
# TODO: Try all reshapings
def reshape_tensor(W):
    "reshape tensor into a maximum rectangle.  Usually the max dimension (N) is the last, so just preserve it"
    dims = W.shape
    N = np.max(dims)
    print(dims)

    M = 1
    for d in dims:
        M = M*d
    M = int(M / N)
    
    if dims[-1]==N:
        Ws = np.reshape(W,(M,N))
    else:
        Ws = np.reshape(W,(N,M))
        
    print(Ws.shape)
    return Ws

### Auto fit MP 

#### Using Kernel Density Estimator

### TODO: float num_spikes, or grid search as int

In [49]:
def marchenko_pastur_fun(x, Q, sigma=1):
    y=1/Q
    
    b=np.power(sigma*(1 + np.sqrt(1/Q)),2) # Largest eigenvalue
    a=np.power(sigma*(1 - np.sqrt(1/Q)),2) # Smallest eigenvalue
    return x, (1/(2*np.pi*sigma*sigma*x*y))*np.sqrt((b-x)*(x-a))

In [50]:
def resid_mp(p, evals, Q, num_spikes=0, bw=0.1, debug=False):  
    "residual that floats sigma but NOT Q or num_spikes YET, 10% cutoff each edge"
    sigma = p

    # kernel density estimator
    kde = KernelDensity(kernel='linear', bandwidth=bw).fit(evals.reshape(-1, 1))
    xde =  np.linspace(0, np.max(evals)+0.5, 1000)
    X_plot =xde[:, np.newaxis]
    log_dens = kde.score_samples(X_plot)
    yde = np.exp(log_dens)   
    
    # MP fit for this sigma
    xmp, ymp = marchenko_pastur_fun(xde, Q=Q, sigma=sigma)
    
    # form residual, remove nan's 
    resid = ymp-yde
    resid = np.nan_to_num(resid)
    
    if debug:
        plt.plot(xde,yde)
        plt.plot(xmp,ymp)
        plt.show()
        print("sigma {}  mean residual {}".format(sigma,np.mean(resid)))

    return resid

In [51]:
def fit_mp(evals, Q):
    "simple fit of evals, only floats sigma right now"
    sigma0=1.0
    [sigma1],cov,infodict,mesg,ierr   = optimize.leastsq(resid_mp, [sigma0], args=(evals, Q), full_output=True)
    return sigma1